# Pattern matching


We will start in the same way as the last notebook started  - by downloading/importing the packages needed and importing the .csv file(s) needed. In this case, we only need the .csv file that has the matched abstracts as we are specifically looking at person-first and identity-first patterns that are "about" autism (or ASD, Asperger's syndrome, etc.). 

We could use the same basic approach to look at person-first and identity-first language for other conditions for which there are good noun and adjective forms of the words (diabetes? obesity? cancer? something else?). Doing that would mean using the .csv file with all of the abstracts or potentially creating and entirely new file of abstracts matched to another condition of interest. However, that lies outside the scope of this research, so I will not address it further here. 

## Get ready 

As always, we start with a couple of code cells that load up and nickname some useful packages, then check file locations, then import files and check them. 

In [1]:
%%capture

# installing necessary pdf conversion packages via pip
# the '%%capture' at the top of this cell suppresses the output (which is normally quite long and annoying looking). 
# You can remove or comment it out if you prefer to see the output. 
!pip install nltk
!pip install spacy -q
!python -m spacy download en_core_web_lg -q


In [2]:
%%capture

import os                         # os is a module for navigating your machine (e.g., file directories).
import nltk                       # nltk stands for natural language tool kit and is useful for text-mining. 
from nltk import word_tokenize    # and some of its key functions
from nltk import sent_tokenize  
tokenizer = nltk.tokenize.punkt.PunktSentenceTokenizer()
from nltk.tokenize import sent_tokenize
nltk.download('punkt')
from nltk.corpus import wordnet                    # Finally, things we need for lemmatising!
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer() 
from nltk.stem.porter import PorterStemmer
porter = PorterStemmer()
nltk.download('averaged_perceptron_tagger')        # Like a POS-tagger...
nltk.download('wordnet')
nltk.download('webtext')
from nltk.corpus import webtext

import pandas as pd
pd.set_option('display.max_colwidth', 200)
import numpy as np
import statistics

#import codecs
import csv                        # csv is for importing and working with csv files

from collections import Counter

import statistics
import re                         # things we need for RegEx corrections
import string 
import spacy 
from spacy.matcher import Matcher 
from spacy.tokens import Span 
from spacy import displacy 
nlp = spacy.load('en_core_web_lg')
nlp.max_length = 1500000 #or any large value, as long as you don't run out of RAM

import math

In [3]:
print(os.listdir("..\\output")  )      

['abstract_count.jpg', 'all_abstracts_no_null_texts.csv', 'matched_abstracts_no_null_texts.csv', 'most_frequent_comparison.csv', 'text_match_results.csv']


## Import

Having checked the contents of the output folder and seen the files we expected to see, we can now import the specific file of interest for this step of the analysis.

In [4]:
matched_texts = pd.read_csv('..\\output\\matched_abstracts_no_null_texts.csv')    # one for just those that match the keyword
len(matched_texts)                                                                # check the length 

906

## Cleaning phase

The first step of pattern matching is to prepare the data by sentence tokenising the text. Like word tokens, sentence tokens become the unit for analysisis. As a trivial example, sentence tokenisation would turn a short text such as 


''' The cat named Cat is one of five cats. Honestly, I wonder why I have so many cats.
''' 

into a list of sentence tokens like

''' [[The cat named Cat is one of five cats.]

[Honestly, I wonder why I have so many cats.]]

''' 

An important difference is that the punctuation within the sentences that contributes to its structured and meaning (e.g. the comma and the full stops) are retained. This punctuation, like the capitalisation at the start of the sentences or for the poper nouns, is also retained as it helps the sentence-tokenisation process identify the words within the sentence correctly for their parts of speech (e.g. which of the words are nouns, verbs, etc. ). 



The sentence tokens are then put on individual rows, filtered to retain only those that contain one or more of the keywords of interest, and then filtered to ensure that there are no empty rows or duplicates. 

In [5]:
sentences  = [sent_tokenize(abstract) for abstract in matched_texts['Text'] ] # tokenize the text column and store as a list
matched_texts['Sentence'] = sentences                                   # copy that list back into df as a new column
sentence_per_row = matched_texts.explode('Sentence')                    # explode column in new df with 1 row/sentence token
len(sentence_per_row)                                                   # check the length of new df


13444

In [6]:
sentence_per_row                                                    # have a look. For the first two rows, 
                                                                    # 'Text' should be same, but 'Sentence' should not

,Unnamed: 0,Title,Session_Code,Authors_and_Affiliations,Text,Year,Email,Author,Affiliations,Sentence
0,59,Genetic defects in sterol metabolism,S60.,F. MoebiusInstitute for Biochemical Pharmacology University of Innsbruck Innsbruck AustriaFabian.M,Genetic defects in sterol metabolizing enzymes have recently emerged asimportant causes of dysmorphogenetic syndromes. They affect enzymesrequired for the removal of methyl groups at C4(NSHDL) th...,2001.0,Fabian.Moebius@uibk.ac.at,NaN,NaN,Genetic defects in sterol metabolizing enzymes have recently emerged asimportant causes of dysmorphogenetic syndromes.
0,59,Genetic defects in sterol metabolism,S60.,F. MoebiusInstitute for Biochemical Pharmacology University of Innsbruck Innsbruck AustriaFabian.M,Genetic defects in sterol metabolizing enzymes have recently emerged asimportant causes of dysmorphogenetic syndromes. They affect enzymesrequired for the removal of methyl groups at C4(NSHDL) th...,2001.0,Fabian.Moebius@uibk.ac.at,NaN,NaN,They affect enzymesrequired for the removal of methyl groups at C4(NSHDL) the shift of the double bond from C8 9to C7 8(3ÃÂ§ sterol D 8 D7isomerase/EBP E.C.
0,59,Genetic defects in sterol metabolism,S60.,F. MoebiusInstitute for Biochemical Pharmacology University of Innsbruck Innsbruck AustriaFabian.M,Genetic defects in sterol metabolizing enzymes have recently emerged asimportant causes of dysmorphogenetic syndromes. They affect enzymesrequired for the removal of methyl groups at C4(NSHDL) th...,2001.0,Fabian.Moebius@uibk.ac.at,NaN,NaN,5.3.3.5) and the removal of the double bond at C7 8(D7 sterol reduc tase/DHCR7 E.C.
0,59,Genetic defects in sterol metabolism,S60.,F. MoebiusInstitute for Biochemical Pharmacology University of Innsbruck Innsbruck AustriaFabian.M,Genetic defects in sterol metabolizing enzymes have recently emerged asimportant causes of dysmorphogenetic syndromes. They affect enzymesrequired for the removal of methyl groups at C4(NSHDL) th...,2001.0,Fabian.Moebius@uibk.ac.at,NaN,NaN,1.3.1.21).
0,59,Genetic defects in sterol metabolism,S60.,F. MoebiusInstitute for Biochemical Pharmacology University of Innsbruck Innsbruck AustriaFabian.M,Genetic defects in sterol metabolizing enzymes have recently emerged asimportant causes of dysmorphogenetic syndromes. They affect enzymesrequired for the removal of methyl groups at C4(NSHDL) th...,2001.0,Fabian.Moebius@uibk.ac.at,NaN,NaN,Missense and nonsense mutations in NSHDL on Xq28 cause X chromosomal dominant CHILD syndrome (congenitalhemidysplasia with ichthyosiform erythroderma and limb defects MIM308050) a rare inborn dis...
...,...,...,...,...,...,...,...,...,...,...
905,2158,NaN,NaN,NaN,P0833Screening for ARX gene mutations is indicated for males problems. The study of this disorder has intensiÜed because with mental retardation associated with West syndrome and/or incidence i...,2004.0,NaN,NaN,NaN,This duplication patients 49 returned with completed metabolic work up.
905,2158,NaN,NaN,NaN,P0833Screening for ARX gene mutations is indicated for males problems. The study of this disorder has intensiÜed because with mental retardation associated with West syndrome and/or incidence i...,2004.0,NaN,NaN,NaN,In this group leads to an expansion of a polyalanine tract in the ARX protein.
905,2158,NaN,NaN,NaN,P0833Screening for ARX gene mutations is indicated for males problems. The study of this disorder has intensiÜed because with mental retardation associated with West syndrome and/or incidence i...,2004.0,NaN,NaN,NaN,"of 49 patients 18 had a diagnosis of autism and 31 of PDD, only one The ARX gene was screened for mutations in Üve males with mental patient was female, average age was 5.4 years."
905,2158,NaN,NaN,NaN,P0833Screening for ARX gene mutations is indicated for males problems. The study of this disorder has intensiÜed because with mental retardation associated with West syndrome and/or incidence i...,2004.0,NaN,NaN,NaN,Sixty percent of the retardation associated with West syndrome and/or dys

In [7]:
matched_sentences = sentence_per_row[sentence_per_row['Sentence'].str.contains('autis|Autis|ASD|Asperger|asperger')]
                                                     # create a new data frame with only the sentences that contain keywords
len(matched_sentences)                               # check the length

2052

In [8]:
matched_sentences = matched_sentences[~matched_sentences['Sentence'].isnull()]  # remove any rows with empty 'Sentence' column
matched_sentences = matched_sentences.drop_duplicates()                         # drop any duplicates
len(matched_sentences)                                                          # check length of remaining data frame

2052

## Extraction

Following the cleaning phase, we move on to the extraction phase. This has two parts, first for the person-first extraction and then for the identity-first extraction. 

The results of both extractions are saved in their own column to make it easy to read and also to allow for a single sentence-token to contain both kinds of patterns. 

### Person-first pattern

In [9]:
pattern_2 = [{"POS": "NOUN"},                                                   # define the person-first pattern(s)
             {'LOWER': 'with'},                                                 # I made 3 for clarity rather than one with 
             {'DEP':'amod', 'OP':"?"},                                          # a real complex regex string
             {'DEP':'amod', 'OP':"?"},
             {'DEP':'amod', 'OP':"?"},
             {"TEXT": {"REGEX": "^[Aa]utism$"}}]

pattern_3 = [{"POS": "NOUN"},
             {'LOWER': 'with'},
             {'DEP':'amod', 'OP':"?"},
             {'DEP':'amod', 'OP':"?"},
             {'DEP':'amod', 'OP':"?"},
             {"TEXT": {"REGEX": "^[Aa]sperger$"}}]

pattern_4 = [{"POS": "NOUN"},
             {'LOWER': 'with'},
             {'DEP':'amod', 'OP':"?"},
             {'DEP':'amod', 'OP':"?"},
             {'DEP':'amod', 'OP':"?"},
             {"TEXT": {"REGEX": "^ASD$"}}]

# Matcher class object 
matcher = Matcher(nlp.vocab)                                                  # define a matcher class object
matcher.add("matching_1", [pattern_2, pattern_3, pattern_4])                  # add my three person-first patterns to it

In [10]:
def find_pattern_match(input):                                               # define a function that applies the person-first
    thingy = nlp(input)                                                       # matcher class object to strings
    match = matcher(thingy)                                                   # and returns any matches to the pattern(s)
    if match == []:
        out_value = ''
    else:
        hold_multi_spans = []
        for match_id, start, end in match:
                string_id = nlp.vocab.strings[match_id]  # Get string representation
                span = thingy[start:end]  # The matched span
                hold_multi_spans.append(span)
        out_value = hold_multi_spans
    return out_value

In [11]:
matched_sentences['Person-first'] = matched_sentences.apply(lambda row: find_pattern_match(row.Sentence), axis = 1)
                                                                        # apply the newly defined person-first matcher function
                                                                        # and store the returned output in a new column
len(matched_sentences)                                                  # double check length remains same

2052

### Identity-first pattern

In [12]:
pattern_a = [{"TEXT": {"REGEX": "^[Aa]utistic"}},                        # do the same for identity-first patterns
             {'DEP':'amod', 'OP':"?"},                                   # again, i wrote 3 patterns for clarity sake
             {'DEP':'amod', 'OP':"?"},
             {'DEP':'amod', 'OP':"?"},
             {"POS": "NOUN"}]

pattern_b = [{"TEXT": {"REGEX": "^[Aa]sperger"}},
             {'DEP':'amod', 'OP':"?"},
             {'DEP':'amod', 'OP':"?"},
             {'DEP':'amod', 'OP':"?"},
             {"POS": "NOUN"}]

pattern_c = [{"TEXT": {"REGEX": "^ASD"}},
             {'DEP':'amod', 'OP':"?"},
             {'DEP':'amod', 'OP':"?"},
             {'DEP':'amod', 'OP':"?"},
             {"POS": "NOUN"}]

# Matcher class object                                         
matcher = Matcher(nlp.vocab) 
matcher.add("matching_2", [pattern_a, pattern_b, pattern_c])            # this overwrites the matcher object to identity-first

In [13]:
matched_sentences['Identity-first'] = matched_sentences.apply(lambda row: find_pattern_match(row.Sentence), axis = 1)
                                                                        # apply the newly overwritten matcher function
                                                                        # and store the returned output in a new column
len(matched_sentences)                                                  # check the length - why not?

2052

### Consolidation

Following the cleaning and extraction phases, the last phase is consolidation. This phase further refines the data by removing all the rows that do not contain a match for one or both of the patterns. For example, there would be a row for "The child was tested for autism." because it contains a keyword of interest. However, this sentence would be eliminated in the consolidation phase as the keyword does not fit into either the person-first or identity-first patterns. 

Further, this phase goes on to lemmatise the extracted patterns so that they can be counted more easily. This phase ends by writing out the consolidated data frame to a .csv for manual inspection. I could not find a feasible way of identifying whether or not the nouns matched in the extraction phase are person-nouns or not. As the list is not a totally unreasonable length (in the hundreds) I found it workable to open in excel, order alphabetically by 'Person-first' and just scan through to check each match manually. I then ordered alphabetically by 'Identity-first' to check those too. As an example, there were quite a few examples of "autistic testing" or "autistic behaviour" that were eliminated because, although they matched the identity-first pattern, they are not about people and so are not example of identity-first language. 

Coincidentally, during this manual checking part of the consolidation phase I learned that, in the context of human genetics research "proband" is a person-noun. 

In [14]:
matched_patterns = matched_sentences[(matched_sentences['Person-first'] != '') | (matched_sentences['Identity-first'] != '')]
                                                     # keep only rows w/ non-null 'Person-first' and/or 'Identity-first' columns
len(matched_patterns)                                # check length

679

In [15]:
matched_patterns = matched_patterns.explode('Person-first')    # explode 'Person-first' column to create 1 row per match
                                                               # if there were two matches within the same sentence
len(matched_patterns)                                          # check the length

681

In [16]:
matched_patterns = matched_patterns.explode('Identity-first')  # Do the same for 'Identity-first' column
len(matched_patterns)                                          # check the length

718

In [17]:
matched_patterns                                               # have a look at them

,Unnamed: 0,Title,Session_Code,Authors_and_Affiliations,Text,Year,Email,Author,Affiliations,Sentence,Person-first,Identity-first
1,150,Location of the first predisposing gene locus for Asperger syndrome on chromosome 1q21 22,C102.,E. Jarvela1 T. Ylisaukko oja2 T. Nieminen3 E. Kempas1 M. Auranen1 L. Peltonen1 1National Public Health Institute Helsinki Finland 2National Public Health Insitute Helsinki Finland 3Uni...,Asperger syndrome (AS) was first described in 1944 by a Viennese physi cian Hans Asperger who reported a group of boys with autistic psychopa thy whose clinical features resembled autism with som...,2001.0,irma.jarvela@hus.fi,NaN,NaN,Asperger syndrome (AS) was first described in 1944 by a Viennese physi cian Hans Asperger who reported a group of boys with autistic psychopa thy whose clinical features resembled autism with som...,,"(Asperger, syndrome)"
1,150,Location of the first predisposing gene locus for Asperger syndrome on chromosome 1q21 22,C102.,E. Jarvela1 T. Ylisaukko oja2 T. Nieminen3 E. Kempas1 M. Auranen1 L. Peltonen1 1National Public Health Institute Helsinki Finland 2National Public Health Insitute Helsinki Finland 3Uni...,Asperger syndrome (AS) was first described in 1944 by a Viennese physi cian Hans Asperger who reported a group of boys with autistic psychopa thy whose clinical features resembled autism with som...,2001.0,irma.jarvela@hus.fi,NaN,NaN,Asperger syndrome (AS) was first described in 1944 by a Viennese physi cian Hans Asperger who reported a group of boys with autistic psychopa thy whose clinical features resembled autism with som...,,"(autistic, psychopa)"
5,443,Preliminary Findings In The Diagnostic Evaluation Of A Sample Of 103 Individuals With Pervasive DevelopmentDisorders,P0341.,E. Steiner M. M. Guerreiro A. P. Marques de Faria Unicamp Campinas B,Pervasive development disorders (PDD) are a heterogeneous group ofneurobehavioral disorders of infancy. In order to identify etiologic factors ina sample of individuals with PDD a total of 103 su...,2001.0,dgmfcm@unicamp.br,NaN,NaN,In order to identify etiologic factors ina sample of individuals with PDD a total of 103 subjects (84 males and 19females) referred for autistic behavior were submitted to a protocol includ ing c...,,"(autistic, behavior)"
7,473,Detection of subtelomeric rearrangements by FISH in patients with idiopathic mental retardation or autism.,P0372.,Ostergaard J. M. Hahnemann H. Hjalgrim K. Nielsen H. M. Nielsen K. Brondum NielsenJohn F. Kennedy Institute Glostrup D,The subtelomeric regions are often involved in chromosomal rearrange 172 Posters: Clinical Genetics and Dysmorphologyments. They are gene rich and therefore rearrangements in these regions are mor...,2001.0,els@kennedy.dk,NaN,NaN,Two groups of patients all with normal standard chromosome analysis were investigated One group consisted of 14 patients with autism andwith at least one other relative affected with autism.,"(patients, with, autism)",
9,538,Clinical cytogenetic and molecular analysis of two families with FRAXE mutation implications for genetic coun selling and prenatal diagnosis,P0442.,I. Tejada1 P. Botella2 M. A. L pez Ar ztegui3 F. Mozo de Rosales4 M. L pez V alverde4 M. L. Onaindia1 M. Molina1 1Genetic Unit Basurto Hospital Bilbao Spain 2Dept. of Paediatrics Txago...,FRAXE full mutations are rare and appear to be associated with mild men tal retardation and with a fragile site in Xq28. This is due to an abnormalexpanded GCC repeat adjacent to a methylated CpG...,2001.0,itejada@hbas.osakidetza.net,NaN,NaN,One of them has an autistic disorder with severe MR an a dysmor phic face similar to FXS.,,"(autistic, disorder)"
...,...,...,...,...,...,...,...,...,...,...,...,...
895,395,NaN,NaN,NaN,"P0107An additional case of the KBG syndrome 1Department of Pediatrics, University of Tartu, Tartu, Estonia, 2TallinnÓs Children B. Demeer1, N. Bednarek2, Z. Belkadi3, M. Khoury3, C. Eschard4, D. ...",2004.0,NaN,NaN,NaN,In developmental delay with speech defect and

In [18]:
Lem = WordNetLemmatizer()                         # Define a short way to call the WordNetLemmatizer

In [19]:
person_lemma_list = []                            # Create an empty list for output of lemmatising the person-first patterns
lemmatized = []                                   # create an empty list to act as a temp variable for the appending process
for phrase in matched_patterns['Person-first']:   # start for loop looking at each pattern in the person-first pattern column
    x = str(phrase)                               # hold the current pattern
    words = x.split()                             # split the current pattern into words
    for word in words :                           # start a for loop looing at each word in the current pattern
        lemword = Lem.lemmatize(word)             # hold a lemmatised copy of the current word
        lemmatized.append(lemword)                # append the lemmatised copy of the current word to the temp variable
    person_lemma_list.append(lemmatized)          # append the temp variable with the lemmatised words to the output list
    lemmatized = []                               # ensure the temp variable is empty

person_lemma_list                                 # have a look at the output list

[[],
 [],
 [],
 ['patient', 'with', 'autism'],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 ['family', 'with', 'autism'],
 [],
 ['individual', 'with', 'autism'],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 ['patient', 'with', 'high', 'functioning', 'autism'],
 ['family', 'with', 'autism'],
 [],
 [],
 [],
 [],
 [],
 [],
 ['individual', 'with', 'idiopathic', 'autism'],
 [],
 [],
 [],
 ['individual', 'with', 'autism'],
 ['patient', 'with', 'autism'],
 ['patient', 'with', 'autism'],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 ['association', 'with', 'autism'],
 [],
 [],
 ['patient', 'with', 'autism'],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 ['family', 'with', 'ASD'],
 [],
 [],
 [],
 ['patient', 'with', 'autism'],
 [],
 ['patient', 'with', 'autism'],
 ['individual', 'with', 'autism'],
 [],
 [],
 [],
 ['child', 'with', 'idiopathic', 'autism'],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 ['p

In [20]:
identity_lemma_list = []                             # repeat the same thing again for identity-first column
lemmatized = []
for phrase in matched_patterns['Identity-first']:
    x = str(phrase)
    words = x.split()
    for word in words :
        lemword = Lem.lemmatize(word)
        lemmatized.append(lemword)
    identity_lemma_list.append(lemmatized)
    lemmatized = []

identity_lemma_list

[['Asperger', 'syndrome'],
 ['autistic', 'psychopa'],
 ['autistic', 'behavior'],
 [],
 ['autistic', 'disorder'],
 ['autistic', 'behavior'],
 ['autistic', 'feature'],
 ['autistic', 'behavior'],
 ['autistic', 'phenotype'],
 ['autistic', 'disorder'],
 ['autistic', 'disorder'],
 ['autistic', 'disorder'],
 ['autistic', 'probands'],
 ['autistic', 'patient'],
 ['autistic', 'patient'],
 ['Asperger', 'syndrome'],
 ['ASDPwith', 'several', 'affected', 'family'],
 ['autistic', 'disorder'],
 ['autistic', 'behavior'],
 ['autistic', 'behavior'],
 ['autistic', 'feature'],
 ['autistic', 'behavior'],
 ['ASD', 'secundum'],
 [],
 ['autistic', 'patient'],
 [],
 ['autistic', 'individual'],
 ['autistic', 'individual'],
 ['autistic', 'group'],
 ['autistic', 'individual'],
 ['autistic', 'spectrum'],
 ['autistic', 'patient'],
 ['Asperger', 'syndrome'],
 ['autistic', 'behaviour'],
 ['autistic', 'behaviour'],
 [],
 [],
 ['autistic', 'feature'],
 ['autistic', 'feature'],
 ['Asperger', 'syndrome'],
 ['autistic', 'b

In [21]:
matched_patterns['Person-first_lemmatized'] = person_lemma_list    # copy the person-first output to new column in data frame 
matched_patterns['Identy-first_lemmatized'] = identity_lemma_list  # copy the identity-first output to new column in data frame 
matched_patterns                                                   # have a look at the data frame with its new columns

,Unnamed: 0,Title,Session_Code,Authors_and_Affiliations,Text,Year,Email,Author,Affiliations,Sentence,Person-first,Identity-first,Person-first_lemmatized,Identy-first_lemmatized
1,150,Location of the first predisposing gene locus for Asperger syndrome on chromosome 1q21 22,C102.,E. Jarvela1 T. Ylisaukko oja2 T. Nieminen3 E. Kempas1 M. Auranen1 L. Peltonen1 1National Public Health Institute Helsinki Finland 2National Public Health Insitute Helsinki Finland 3Uni...,Asperger syndrome (AS) was first described in 1944 by a Viennese physi cian Hans Asperger who reported a group of boys with autistic psychopa thy whose clinical features resembled autism with som...,2001.0,irma.jarvela@hus.fi,NaN,NaN,Asperger syndrome (AS) was first described in 1944 by a Viennese physi cian Hans Asperger who reported a group of boys with autistic psychopa thy whose clinical features resembled autism with som...,,"(Asperger, syndrome)",[],"[Asperger, syndrome]"
1,150,Location of the first predisposing gene locus for Asperger syndrome on chromosome 1q21 22,C102.,E. Jarvela1 T. Ylisaukko oja2 T. Nieminen3 E. Kempas1 M. Auranen1 L. Peltonen1 1National Public Health Institute Helsinki Finland 2National Public Health Insitute Helsinki Finland 3Uni...,Asperger syndrome (AS) was first described in 1944 by a Viennese physi cian Hans Asperger who reported a group of boys with autistic psychopa thy whose clinical features resembled autism with som...,2001.0,irma.jarvela@hus.fi,NaN,NaN,Asperger syndrome (AS) was first described in 1944 by a Viennese physi cian Hans Asperger who reported a group of boys with autistic psychopa thy whose clinical features resembled autism with som...,,"(autistic, psychopa)",[],"[autistic, psychopa]"
5,443,Preliminary Findings In The Diagnostic Evaluation Of A Sample Of 103 Individuals With Pervasive DevelopmentDisorders,P0341.,E. Steiner M. M. Guerreiro A. P. Marques de Faria Unicamp Campinas B,Pervasive development disorders (PDD) are a heterogeneous group ofneurobehavioral disorders of infancy. In order to identify etiologic factors ina sample of individuals with PDD a total of 103 su...,2001.0,dgmfcm@unicamp.br,NaN,NaN,In order to identify etiologic factors ina sample of individuals with PDD a total of 103 subjects (84 males and 19females) referred for autistic behavior were submitted to a protocol includ ing c...,,"(autistic, behavior)",[],"[autistic, behavior]"
7,473,Detection of subtelomeric rearrangements by FISH in patients with idiopathic mental retardation or autism.,P0372.,Ostergaard J. M. Hahnemann H. Hjalgrim K. Nielsen H. M. Nielsen K. Brondum NielsenJohn F. Kennedy Institute Glostrup D,The subtelomeric regions are often involved in chromosomal rearrange 172 Posters: Clinical Genetics and Dysmorphologyments. They are gene rich and therefore rearrangements in these regions are mor...,2001.0,els@kennedy.dk,NaN,NaN,Two groups of patients all with normal standard chromosome analysis were investigated One group consisted of 14 patients with autism andwith at least one other relative affected with autism.,"(patients, with, autism)",,"[patient, with, autism]",[]
9,538,Clinical cytogenetic and molecular analysis of two families with FRAXE mutation implications for genetic coun selling and prenatal diagnosis,P0442.,I. Tejada1 P. Botella2 M. A. L pez Ar ztegui3 F. Mozo de Rosales4 M. L pez V alverde4 M. L. Onaindia1 M. Molina1 1Genetic Unit Basurto Hospital Bilbao Spain 2Dept. of Paediatrics Txago...,FRAXE full mutations are rare and appear to be associated with mild men tal retardation and with a fragile site in Xq28. This is due to an abnormalexpanded GCC repeat adjacent to a methylated CpG...,2001.0,itejada@hbas.osakidetza.net,NaN,NaN,One of them has an autistic disorder with severe MR an a dysmor phic face similar to FXS.,,"(autistic, disorder)",[],"[autistic, disorder]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
895,395,NaN,NaN,NaN,"P0107An additional case of the KBG syndrome 1Department of Pediatrics, Univ

In [22]:
matched_patterns.to_csv('..\\output\\pattern_matches_to_review.csv')        
                                                            # Write the data frame to a .csv for manual processing in excel

## Chart person first or identity first by year

In [ ]:
person_identity_first = pd.read_csv('..\\..\\2023_Second_analysis\\output\\text_match_results.csv')
person_identity_first = person_identity_first.dropna(how='all')
person_identity_first['Year'] = person_identity_first['Year'].astype('Int64')

In [ ]:
person_count = person_identity_first.groupby(['Year'])['Person-first'].count()
identity_count = person_identity_first.groupby(['Year'])['Identity-first'].count()


In [ ]:
person_identity_count=pd.concat([person_count,identity_count],axis=1)


In [ ]:
person_identity_count.plot()
plt.show()

In [ ]:
person_examples = person_identity_first.groupby(['Person-first'])['Person-first'].count()
identity_examples = person_identity_first.groupby(['Identity-first'])['Identity-first'].count()


In [ ]:
person_identity_examples=pd.concat([person_examples,identity_examples],axis=1)


In [ ]:
person_identity_examples.sort_values(by=['Person-first'], ascending=False)

In [ ]:
person_identity_examples.sort_values(by=['Identity-first'], ascending=False)

In [ ]:
person_identity_examples.notnull().sum()

In [ ]:
has_person = person_identity_first[~person_identity_first['Person-first'].isnull()]
len(has_person)


In [ ]:
has_identity = person_identity_first[~person_identity_first['Identity-first'].isnull()]
len(has_identity)


## Count abstracts by the structures they use

In [ ]:
person_by_title = person_identity_first.groupby(['Title'])['Person-first'].count()
identity_by_title = person_identity_first.groupby(['Title'])['Identity-first'].count()
title = pd.concat([person_by_title,identity_by_title],axis=1)
title

In [ ]:
title.sort_values(by=['Identity-first'], ascending=False)

In [ ]:
title.sort_values(by=['Person-first'], ascending=False)

In [ ]:
columns = ['Person-first','Identity-first']
filter_ = (title[columns] > 0).all(axis=1)
title[filter_]
len(title[filter_])


In [ ]:
title[filter_].sort_values(by=['Person-first'], ascending=False)

In [ ]:
title[filter_].sort_values(by=['Identity-first'], ascending=False)

## Word counts by part of speech


In [ ]:
POS_p_i = []

for token in p_i_doc:
    this_token = [token.text, token.lemma_, token.pos_, token.tag_]
    if any (s in token.text for s in ['autistic', 'Autistic', 'autism', 'Autism', 'ASD', 'asd', 'Asperger', 'asperger']):
        POS_p_i.append(this_token)

In [ ]:
with open('..\\counts\\ESHG\\POS.csv', "w", encoding='utf8') as outfile:
        write = csv.writer(outfile)
        for item in POS_p_i:
            write.writerow([item])

In [ ]:
p_f_lower = [word.lower() for word in person_first]     # make those tokens lowercase
p_f_no_punct = [w.translate(table_punctuation) for w in p_f_lower] # remove the punctuation
p_f_no_space = (list(filter(lambda x: x, p_f_no_punct)))           # remove any extra whitespace

In [ ]:
#for saving output
os.makedirs('folder/subfolder', exist_ok=True)  
df.to_csv('folder/subfolder/out.csv') 